In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

import pickle
import random
from string import punctuation

import gensim
from gensim import corpora

import spacy
from spacy.lang.en import English

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/katieta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# tokenizing/lemma-tizing stuff

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
#         elif token.orth_.startswith('#'):
#             lda_tokens.append('HASHTAG')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def get_text_data(corpus, token_len=20):
    text_data = []
    for i in range(len(corpus)):
        if i % 20000 == 0:
            print("{} / {}".format(i, len(corpus)))
#             if i > 0:
#                 print(text_data[i - 1])
        try:
            tokens = prepare_text_for_lda(corpus[i])
            while len(tokens) < token_len:
                tokens.append('STOP')
                
            if len(tokens) > token_len:
                tokens = tokens[:token_len - 1]
                tokens.append('STOP')
                text_data.append(tokens)
            else:
                text_data.append(tokens)
        except Exception as e:
            print(e)
    print("finished")
    return text_data

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
# print(stoplist)
en_stop = set(nltk.corpus.stopwords.words('english'))
# print(stoplist)

def get_dict_corpus(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2idx(text) for text in text_data]
    return dictionary, corpus

In [93]:
dem_tweets = pd.read_csv('../data/tweet_data/dem_tweets_cleaned.csv', index_col=False) # dem tweets w/ uncontested candidates removed
dem_tweets['party'] = [0 for i in range(len(dem_tweets))]
rep_tweets =pd.read_csv('../data/tweet_data/rep_tweets_cleaned.csv', index_col=False) # rep tweets w/ uncontested candidates removed
rep_tweets['party'] = [0 for i in range(len(rep_tweets))]
all_tweets_no_topics = pd.concat([dem_tweets, rep_tweets])


all_tweets = pd.read_csv('../data/tweets_with_topics.csv', index_col=False) # all tweets with labeled topics



In [95]:
all_tweets.head(3)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax,fake_news,environment
0,0,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0,0,0
1,1,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0,0,0
2,2,here is an important notice concerning christo...,0,0,0,0,0,0,0,0,0,0,0


In [89]:
# all_tweets['stripped_text'] = all_tweets['stripped_text'].str.lower()
# all_tweets = all_tweets_topics

In [177]:
gun_control_keywords = [' gun ', 'gun ', 'gun control', 'guncontrol', 'gun reform', 'gunreformnow', 'nra'
                           ,'2nd amendment',  'assault weapon', '2a', 'firearm', 'tree of life', 'columbine'
                            , 'marchforourlives', 'shotgun', 'second amendment', 'gunsense', 'gunsensenow',
                       'momsdemand', 'gunviolence', 'wearorange', 'gunreform', 'semi-automatic', 'assault weapon'
                       , 'assault-type weapons']

healthcare_keywords = ['healthcare', 'health care', 'medicare', 'medicaid', 'health-care', 'affordable care act', 'aca ']

abortion_keywords = ['prolife', 'pro-life', 'pro life', 'prochoice', 'pro-choice', 'pro choice', 'reproductive'
                     , 'abortion', 'roe45', ' roe ']

immigration_keywords = ['immigration', 'borders', 'secure border', 'build wall', 'build a wall', 'build the wall'
                       , 'buildthewall', 'immigrant']

kavanaugh_keywords = ['kavanaugh']
education_keywords = ['education']
jobs_keywords = ['jobs', 'wages', 'workers']
tax_keywords = ['taxes', 'tax cut']
fake_news_keywords = ['fake news']
environment_keywords = ['global warming', 'clean energy', 'plastic pollution'
                       , 'renewable energy', 'earthday', 'coastline', 'renewable', 'solar', 'climatechange', 'carbon', 'fossil']
women_keywords = ['women\'s rights', 'sexual assault', 'metoo']

def label_topic(tweet, keywords):
    for kw in keywords:
        if kw in tweet:
            return 1
    return 0

def re_label_topic(df, topic, keywords):
    for i, r in df.iterrows():
        if r[topic] == 0:
            r[topic] = label_topic(r['stripped_text'], keywords)
    
# all_tweets['gun_control'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, gun_control_keywords))
# all_tweets['health_care'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, healthcare_keywords))
# all_tweets['abortion'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, abortion_keywords))
# all_tweets['immigration'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, immigration_keywords))
# all_tweets['kavanaugh'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, kavanaugh_keywords))
# all_tweets['education'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, education_keywords))
# all_tweets['jobs'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, jobs_keywords))
# all_tweets['tax'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, tax_keywords))
# all_tweets['fake_news'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, fake_news_keywords))
all_tweets['environment'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, environment_keywords))
# all_tweets['women'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, women_keywords))

# for k in environment_keywords:
#     all_tweets.loc[all_tweets['stripped_text'].str.contains(k), 'environment'] = 1


In [178]:
# all_tweets = all_tweets.drop(labels=['fake news'], axis=1)
all_tweets.head(5)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax,fake_news,environment,sexual_assault,gender,women
0,0,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,0
1,1,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,0
2,2,here is an important notice concerning christo...,0,False,0,0,0,0,0,0,0,0,0,False,False,0
3,3,i believe that we need to find real solutions ...,0,False,0,0,0,0,0,0,0,0,0,False,False,0
4,4,"it's time to turn the page on alabama's past, ...",0,False,0,0,0,0,0,0,0,0,0,False,False,0


# START HERE 

In [179]:
topic = 'environment'
# all_tweets = all_tweets_topics

print(len(all_tweets[all_tweets[topic] == 1]))
topic_tweets = pd.concat([all_tweets[all_tweets[topic] == 1], all_tweets[all_tweets[topic] == 0].sample(len(all_tweets[all_tweets[topic] == 1]))])
print(len(topic_tweets))
full_tweets = pd.concat([topic_tweets, all_tweets[~all_tweets['tweet_idx'].isin(topic_tweets['tweet_idx'].unique())]])

print(len(all_tweets))
print(len(full_tweets))
# 

4577
9154
557549
557549


In [180]:
for t in topic_tweets[topic_tweets[topic] == 1].sample(100)['stripped_text'].values:
    print(t, '\n')

my apologies as i never saw the email. as you know i left my career 5 years ago and invested my savings in @helicaloutpost and my platform advocates 100% renewables. hopefully my actions mean more than my failure to see and respond to email. happy earth day!!!  

yes, i did. i noted that the victories that environmental progress & our allies achieved required compromising and giving solar a grotesquely large subsidy to save nuclear plants. making a political compromise is different from being deceptive about how solar/wind lock-in fossil. 

sure, those contribute, but that doesn't change the fact that nrdc is using renewables to greenwash its advocacy of replacing nuclear with fossil fuels 

campaign life is not too shabby today. does this look like a place we want oil rigs, barges, and pipelines when there are clean, renewable alternatives that would secure our long-term energy independence? #protectourcoast #dontdrillnc #iheartnc7  

47. as such  while there might be good reasons to 

In [181]:
# tokenize data 

model_token_data = get_text_data(full_tweets['stripped_text'].values)
# dictionary, corpus = get_dict_corpus(model_token_data)
# print(len(dictionary))
X = [{w: 1 for w in x} for x in model_token_data]
print(X[0])
dv = DictVectorizer()
X = dv.fit_transform(X)

print(X.shape)
# print(X.todense())
names = dv.get_feature_names()

0 / 557549
20000 / 557549
40000 / 557549
60000 / 557549
80000 / 557549
100000 / 557549
120000 / 557549
140000 / 557549
160000 / 557549
180000 / 557549
200000 / 557549
220000 / 557549
240000 / 557549
260000 / 557549
280000 / 557549
300000 / 557549
320000 / 557549
340000 / 557549
360000 / 557549
380000 / 557549
400000 / 557549
420000 / 557549
440000 / 557549
460000 / 557549
480000 / 557549
500000 / 557549
520000 / 557549
540000 / 557549
finished
{'retweeted': 1, 'bruce': 1, 'munger': 1, 'SCREEN_NAME': 1, 'thursdaythoughts': 1, 'alpolitics': 1, 'wednesdaywisdom': 1, 'aldems': 1, 'bluetsunami': 1, 'climatechange': 1, 'democrat': 1, 'election2018': 1, 'firstdayofsummer': 1, 'humanrights': 1, 'lgbtvoices': 1, 'lgbtq': 1, 'STOP': 1}
(557549, 122474)


In [163]:
print(len(topic_tweets))
topic_subset = X[:len(topic_tweets)]

X_train, X_test, y_train, y_test = train_test_split(topic_subset, topic_tweets[topic].values[:len(topic_tweets)], test_size=0.1)

new_file = True
if os.path.isfile('pickles/{}_logistic_regr.pkl'.format(topic)):
    with open('pickles/{}_logistic_regr.pkl'.format(topic), 'rb') as f:
        clf = pickle.load(f)
    new_file = False
else:
    clf = LogisticRegression()
    
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

# save model
with open('pickles/{}_logistic_regr.pkl'.format(topic), 'wb') as f:
    pickle.dump(clf, f)

for w, n in sorted(zip(clf.coef_[0], names), key=lambda e:e[0], reverse=True):
    print(w, n)

13362
7.386342776766144 climate
7.1373041086374505 climatechange
6.964031738157461 solar
5.970113753161998 fossil
5.688174743474094 carbon
5.339403872087395 earthday
4.957436747332172 renewables
4.890233151677226 carbondale
4.556860522839186 renewable
4.3007739743805615 energy
4.287288869371037 coastline
4.244193266493665 climatechangeisreal
4.204542639644127 renewableenergy
3.701951855838623 earthday2018
3.316062416326491 clean
3.2773207413967964 warming
3.201461218041972 nofossilfuelmoney
2.8364160064810497 global
2.66350429694572 offfossilfuels
1.864088367640307 earthday18
1.8412087804328552 change
1.7194505450017161 fossilfree
1.676771302791067 decarbonize
1.6760001027030726 earth
1.576881524179584 nuclear
1.5723283170474232 carbone
1.5383826746232288 medicareforall
1.5253974924004272 pledge
1.5020161603865132 plant
1.408986867145423 climateresistance
1.3863505244869976 planet
1.3721132865489007 environment
1.207175219807873 green
1.1767016517744497 drilling
1.1684044430299956 happ

0.217121005109115 trump
0.21692518566658936 arguing
0.21675887096992097 close
0.21638340403474554 notice
0.21627998690043476 turntexasblue
0.2160610239518457 arrive
0.21600497805802163 reliance
0.2154803249178875 ofasacramento
0.2152166693554224 uranium
0.21437251424308595 racism
0.2136938434752612 reenter
0.21355559984124553 neutral
0.21324950358930472 own
0.21304264905341241 common
0.21268934646606177 ohioan
0.21255706954638062 words
0.2118689598814666 describe
0.21104146889585265 mixology
0.21103655103459867 equality
0.21082484120448108 goal
0.21011466950972257 teammorse
0.20939092016014338 behold
0.20937964192064462 cleanair
0.20859174732303368 respond
0.20839217028401064 basic
0.20814669149916556 drill
0.20812279386683696 bringing
0.20799874386736053 likely
0.20776394920059435 arabia
0.20711224319574242 redemption
0.20671686767720265 biological
0.20643649759318225 freetheinternet
0.20561758243619363 productivity
0.20525808351676864 heavily
0.20466137550383665 threat
0.204594809751

0.11910546603523425 relate
0.11905053969635869 singlepayer
0.11872960211071548 evelyn
0.11866269428517236 fort
0.11866269428517236 getoutdoors
0.11860531907957456 expression
0.11860531907957456 provocative
0.11850726592679489 zone
0.1184651146740852 means
0.11801122232512913 panic
0.11795860583811302 outside
0.11795696157787353 think
0.11790828742543634 begin
0.1178762800946482 progress
0.1175044116332173 hours
0.11739405268571337 glacier
0.11714714383556017 unconscionable
0.11690917833249427 thefamilyguy
0.11690917833249427 theoutdoorguy
0.1167231423270305 albuquerque
0.11654792532947535 sisolak
0.11625309193744814 turbine
0.11619831969014797 strength
0.11617748215651261 blood
0.11616692801503357 imwithkim
0.11616692801503357 seuss
0.11610807072820763 intervention
0.1154610230004746 state
0.11510168513865963 teamsisolak
0.11504920853586703 votewithme2018
0.11479661707232203 steward
0.11471423416179989 innovative
0.11419825618083325 industrial
0.1140672585734559 upcoming
0.113910440677

0.04392915031092119 shareholder
0.0439025639648996 coingate
0.0439025639648996 lending
0.0438774657272508 failing
0.04385702735568676 former
0.04382738418944834 booster
0.04382462261485444 drastic
0.04382210373412571 removal
0.043814448557558665 clarity
0.043797648311570694 buddy
0.043540809982436715 resort
0.04347284031160807 daily
0.04335745862414909 overregulation
0.043235685241814564 harriethagemanforgovernor
0.043235685241814564 housingwhich
0.043235685241814564 inhibit
0.04322036297822909 gerhard
0.04322036297822909 votersappreciate
0.04321129807762841 pretending
0.04315090561016467 spin
0.04315090561016467 transcanada
0.04298175487041638 narrow
0.04293748701993831 hurricaneflorence
0.042928528259086676 cleanrenewableenergy
0.04290020799334259 stormsurge
0.04285792002503632 ballroom
0.042812890461919495 effectiveness
0.04276241119451973 dreamact
0.04272943170834477 .@votejohnwarren
0.04272825649511603 passionate
0.04264533284081528 complement
0.04255981656653499 front
0.042419846

0.02007205386258872 slap
0.020067496165804312 owe
0.020061947413890304 departed
0.020023594099119038 annoy
0.020022537852007874 140,000
0.020022537852007874 16,200
0.020022537852007874 328,000
0.020022537852007874 59,400
0.020002339888824237 orbit
0.019970701331249775 unsolvable
0.019962232015866405 fellow
0.019919501784217208 implication
0.019875388580999773 hefty
0.019756217075074696 dependence
0.019754731497765236 sanantonio
0.019750493006105728 massive
0.019735811391331758 economically
0.019733276606065706 warren
0.01971539416212615 bumper
0.019694909005285925 beats
0.01969330898761256 equinox
0.01966863292360494 politicisizing
0.01958516326690523 substance
0.01952967519983613 array
0.019528102439851588 issas
0.01949138877536645 bluster
0.01948862490435642 superfund
0.01948268933303735 disastermitigation
0.019384334006199194 24hoursofreality
0.019384334006199194 callcongressonclimate
0.019384334006199194 chiclimateforum
0.019382562164421677 alexd2018
0.019356258563273145 tidal
0.01

...

-0.027480619856644805 gloss
-0.027493666597701617 friedman
-0.027518450095352243 freak
-0.02752289110848334 perish
-0.027527233405866998 obese
-0.027564462108043756 jughandle
-0.027564462108043756 welle
-0.027568154502798092 betosaveourparty
-0.027568154502798092 firemaxey
-0.02757018542681094 dishonest
-0.027600669362727245 defendthewest
-0.027610278824013425 fakemoderates
-0.02761316797134342 statistics
-0.027613416406833593 smear
-0.027621832744789812 sasse
-0.027622140679886786 christianity
-0.027680439308688486 landline
-0.027704437788596027 example
-0.027710935217770898 breed
-0.027710968807254443 hotline
-0.02773594738111884 stripper
-0.027738592866843093 whoop
-0.027745048425177898 fairfax
-0.027745424490089692 protector
-0.027768167595946594 sumner
-0.027775186312698527 apache
-0.027775186312698527 cowboy
-0.027775186312698527 pawnee
-0.02777628821842459 walkill
-0.02777663249225199 eruption
-0.027779278127626594 reconciliation
-0.027796187287215966 nationalgunviolenceawarenes

-0.03549585562859833 honduras
-0.03550935867289354 plunge
-0.03552635957621488 currency
-0.03553029868291032 separation
-0.035537108159274515 regulatory
-0.03554122698122598 murieta
-0.03555134095005702 chesterfield
-0.03555134095005702 lewandowski
-0.0355810373673802 4-year
-0.035598922369700865 gregory
-0.03560367384477195 jackie
-0.0356156433434249 sovereignnation
-0.035641132163656435 remembertomschweich
-0.035644574378840894 electronic
-0.03566193449076858 lately
-0.03570900811363105 stride
-0.03573829229020945 monthan
-0.035747048648848576 mergeforpower
-0.03576766160866143 funston
-0.03576766160866143 stinson
-0.03576766160866143 woods
-0.03579461334851279 teammarsha
-0.03582453490424227 isupportjamesandmandybecause
-0.03586011143643613 chavez
-0.03586011143643613 harkey
-0.035899986161828816 tienen
-0.03590078251826112 lincoln
-0.035905677648369806 oppression
-0.03590711826281476 medal
-0.03594327334751925 employ
-0.03594490523505869 catering
-0.03597630436925218 gohmert
-0.035

-0.05217551904439081 sophisticate
-0.05220172685364867 miami
-0.05221512929603526 leeds
-0.05222183048653715 hillsborough
-0.052276021256803085 redistricting
-0.052302737384663914 prohibition
-0.052355891779193 sexually
-0.05236935128068426 democrats2018
-0.052375464313853544 papers
-0.052376613115720366 finances
-0.052470750265740086 homeland
-0.05253546161065716 chicago
-0.052554015541130504 protectourcare
-0.05255431838015951 knowing
-0.05261993087278108 simultaneously
-0.052659596046631 anyway
-0.05267397675826355 fascinate
-0.05268140938089585 cagov
-0.05269878676353296 astronaut
-0.05269878676353296 princess
-0.05269878676353296 tiara
-0.05270286504289843 mutual
-0.052762009175841694 problematic
-0.05276561991439312 mnsen
-0.0527686553243697 carve
-0.05280878181835679 scotland
-0.05283678136078087 litany
-0.05284922151569984 fixpa
-0.05288431861693114 despair
-0.05289057197770537 ottawa
-0.052896490926761255 clinton
-0.05291639726443391 hire
-0.05292172784074581 clark
-0.05296090

-0.07889379034428332 granite
-0.07889976402886209 basis
-0.07891646284339116 insurer
-0.07895939788635038 bidding
-0.07896465412805277 opiate
-0.07899251932694734 falter
-0.07902082033900083 offender
-0.07904224207964348 adirondack
-0.07904879405507818 seventeen
-0.07915029272149263 childhood
-0.0792508416008972 masses
-0.07927806481206463 sicken
-0.07931599017601072 harris
-0.07954495426365549 anybody
-0.07964907074854877 transparency
-0.07966773862624525 findyourpark
-0.07973082133877826 bible
-0.07977636918317701 15/hr
-0.07982216306517957 site
-0.07996546873926577 constantly
-0.07996578274741502 race
-0.0800233380202735 jersey
-0.0800565569712437 friendly
-0.08021022945492093 anymore
-0.0802122128827403 competition
-0.08024315847698664 mine
-0.08035136946182622 officially
-0.08050049195003561 rubberstamp
-0.08050893822581832 fascist
-0.08052256274892293 defunding
-0.08065527474990682 ownership
-0.08066562063692626 midwest
-0.08068751236321478 takeitback
-0.08069047797037178 disclos

-0.1813304972798193 transit
-0.18167932711441181 kennedy
-0.18168308826185817 anything
-0.18176746218587275 schedule
-0.1819319964806026 attend
-0.1819426219428383 home
-0.1820332701590482 watch
-0.1821282376869115 prosper
-0.18238346359815588 turnout
-0.18246098540095113 magical
-0.18262314507333732 client
-0.18273902319833807 domestic
-0.18286742084058177 thrill
-0.1829925199952967 nevadan
-0.1832343616549035 continuous
-0.18371948483252673 canvass
-0.18411639512877456 preservation
-0.1841653337657184 accessibility
-0.18432913555561495 cent
-0.1846023909671608 simply
-0.18466236675402453 servant
-0.1849390489506937 poison
-0.18504553352787687 rolling
-0.18512428849874743 mtsen
-0.18536849346969617 association
-0.1859814876508783 intern
-0.18605739036770347 sharing
-0.1861252980380462 positive
-0.18620000821746693 challenge
-0.1864239143218424 google
-0.18668937412298148 differently
-0.18708655615835285 governor
-0.18709608642176825 content
-0.18730330949943064 bluff
-0.18752310078274

In [164]:
c = 0
for i in y_train:
    if i == 1:
        c += 1 
print(c)
print(len(y_train))

6002
12025


In [165]:
score

0.9820493642483171

## Run Predictions

In [166]:
pred_probs = clf.predict_proba(X[len(topic_tweets) + 1:])
pred_probs[0]

array([0.99233533, 0.00766467])

In [167]:
predictions = clf.predict(X[len(topic_tweets) + 1:])
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [174]:
c = 0
t = 0
tw = []
below_threshold = 0
tweets = full_tweets['stripped_text'].values[len(topic_tweets) + 1:]
full_tweets_idx = full_tweets['tweet_idx'].values[len(topic_tweets) + 1:]
idx = []
missed_tweets = []
tweets_above_threshold = []
for i in range(len(pred_probs)):
    if pred_probs[i][1] >= 0.96:
        t += 1
        idx.append(full_tweets_idx[i])
        tweets_above_threshold.append(tweets[i])
        
    else:
        below_threshold += 1
        
print("num above threshold: ", len(tweets_above_threshold)) # total above threshold
print("below threshold: ", below_threshold)

num above threshold:  262
below threshold:  543924


In [176]:
for t in np.random.choice(tweets_above_threshold, min(len(tweets_above_threshold), 100)):
    print(t, '\n')

feels kind of like our political climate right now in america ...  

thanks @mikebloomberg and @jerrybrowngov for serving as unofficial us delegation at paris climate talks #cop23bonn  

fmr. sen @ricksantorum telling our youth to learn cpr instead of asking lawmakers for action is appalling and illustrates the dissociative climate in washington. the cure for washingtons cardiac arrest will be administered in november. #vote2018 

11. since its birth in the 1950s, the nuclear industry & scientific community have stressed the separateness of energy production and weapons. but recent statements by middle eastern leaders have thrust the connections  technical, workforce, & motivational  into the limelight. 

erratic wind turbines can not replace reliable nuclear plants, warns swedens electricity grid operator, who apparently didnt get the we need an energy mix! memo  

philippines considers starting up mothballed nuclear plant to save money & reduce coal-burning with the need for cheaper,

In [147]:
# label the tweets above threshold with a 1
# all_tweets.loc[all_tweets['tweet_idx'].isin(idx), topic] = 1

all_tweets.to_csv('../data/tweets_with_topics.csv', index=False)

In [569]:
all_tweets.head(10)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax
0,0,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0
1,1,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0
2,2,here is an important notice concerning christo...,0,0,0,0,0,0,0,0,0
3,3,i believe that we need to find real solutions ...,0,0,0,0,0,0,0,0,0
4,4,"it's time to turn the page on alabama's past, ...",0,0,0,0,0,0,0,0,0
5,5,there is hope for alabama. this inspiring vide...,0,0,0,0,0,0,0,0,0
6,6,live,0,0,0,0,0,0,0,0,0
7,7,join us for a special and important webcast co...,0,0,0,0,0,0,0,0,0
8,8,countryman isn't standing down...his fight con...,0,0,0,0,0,0,0,0,0
9,9,please join us for a very special 2018 alabama...,0,0,0,0,0,0,0,0,0


In [59]:
topics = ['gun_control', 'health_care', 'abortion', 'immigration', 'kavanaugh', 'education', 'jobs', 'tax']

unlabeled_tweets = all_tweets[(all_tweets['gun_control'] == 0) 
           & (all_tweets['health_care'] == 0)
          & (all_tweets['abortion'] == 0)
          & (all_tweets['immigration'] == 0)
          & (all_tweets['kavanaugh'] == 0)
          & (all_tweets['education'] == 0)
          & (all_tweets['jobs'] == 0)
          & (all_tweets['tax'] == 0)]

In [60]:
print(len(unlabeled_tweets))
print(len(unlabeled_tweets) / len(all_tweets))
print('\n')
for t in unlabeled_tweets.sample(10)['stripped_text'].values:
    print(t, '\n')

485470
0.8707216764804528


i have been trying to make sure i follow #il14 accounts. if i have missed you, let me know. yours are the voices i want to hear first & after elected, you come with me as my only special interest group. your political affiliation does not matter because we are in this together. 

my running mate @embry4md on @c4show talking about the need for the state to be a true partner with baltimore city in order to reduce crime. 

7) cut the red-tape slowing green energy transition; strong green action i. $52 billion for northern california wind turbines ii. end corrupt regulations that limit residential solar iii. build reservoirs & aquifer restoration in south central valley 

march for our lives - spokane march 24, 2018 - 12 pm rally, 1 pm march riverfront park clock tower facebook event:   

#mtgopvictorytour #maga #mtsen #mtpol  

henry martin 2018 is working hard to beat @repsamgraves here in mo6! support your local candidates and let's take back congress! 

yea.